# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [2]:
# Download link 1
!gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails) 
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb
To: /content/hw7_data.zip
100% 9.57M/9.57M [00:00<00:00, 252MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          
Mon May  2 13:11:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version     of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.1 MB 14.9 MB/s 
     |████████████████████████████████| 880 kB 50.6 MB/s 
     |████████████████████████████████| 3.3 MB 77.7 MB/s 
     |████████████████████████████████| 96 kB 7.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.50-py3-none-any.whl size=895166 sha256=778bc149c9bc1b15b9f749696a3f5c2bf7bc4704d10030a03137206546b6689e
  Stored in directory: /root/.cache/pip/wheels/d9/72/54/519f0d5143cc6c73fa3297509123c86fc8586a7fdea8d25311
Successfully built sacremoses
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.0.0 which is incompatible.


## Import Packages

In [1]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [2]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
# GPU 在不需要這麼高精度的運算時可以轉換成 float_16（加速）
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Model and Tokenizer




 

In [4]:
model_1 = BertForQuestionAnswering.from_pretrained("/content/drive/MyDrive/ml_hw8/luhua_macbert_82775").to(device)
model_2 = BertForQuestionAnswering.from_pretrained("/content/drive/MyDrive/ml_hw8/luhua_macbert_83107").to(device)
model_3 = BertForQuestionAnswering.from_pretrained("/content/drive/MyDrive/ml_hw8/luhua_macbert_83017").to(device)
model_4 = BertForQuestionAnswering.from_pretrained("/content/drive/MyDrive/ml_hw8/luhua_macbert_82331").to(device)
model_5 = BertForQuestionAnswering.from_pretrained("/content/drive/MyDrive/ml_hw8/luhua_macbert_83461").to(device)
model_6 = BertForQuestionAnswering.from_pretrained("/content/drive/MyDrive/ml_hw8/luhua_macbert_sixth").to(device)

tokenizer = BertTokenizerFast.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [5]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [6]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False, return_offsets_mapping=True)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False, return_offsets_mapping=True)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False, return_offsets_mapping=True)
# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

dev_paragraphs = [i.replace(' ','✔').replace('\u200b','✦').replace('\u200e', '☺').replace('\u3000', '☆').replace('#','●') for i in dev_paragraphs]
test_paragraphs = [i.replace(' ','✔').replace('\u200b','✦').replace('\u200e', '☺').replace('\u3000', '☆').replace('#','●') for i in test_paragraphs]


## Dataset and Dataloader

In [7]:
import random

class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 384
        self.counter = 0
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 64

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]
        self.counter = self.counter + 1
        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        # input_id, token, attention_mask
        # 每次 random 一個數字當作 offset
        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])
            # A single window is obtained by slicing the portion of paragraph containing the answer
            offset = random.randint(0, 383-(answer_end_token - answer_start_token))
            # 在 0（起始）與 
            # mid 往左二分之一的最大 paragraph 長度（一般情況）、總長-最大 paragraph（最後）=> 兩者較小者
            # 上述兩者中取最大值
            if(self.counter % 2) == 0:
              paragraph_start = max(0, min(answer_start_token - (offset), len(tokenized_paragraph) - self.max_paragraph_len))
              paragraph_end = min(paragraph_start + self.max_paragraph_len, len(tokenized_paragraph))
            else:
              paragraph_end = min(len(tokenized_paragraph), max(answer_end_token + (offset), 0 + self.max_paragraph_len))
              paragraph_start = max(0, paragraph_end - self.max_paragraph_len)
            # print(paragraph_start, answer_start_token, answer_end_token, paragraph_end)

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask


train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

In [8]:
def evaluate(data, output_1, output_2, output_3, output_4, output_5, output6, paragraph_index, doc_stride, is_test, n_best):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    # check 是否存在 start index 比 end index 更後面的問題

    # 

    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    start_index_in_paragraph = 0
    end_index_in_paragraph = 0
    best_of_n_best_prob = -np.inf
    start_logits = (output_1.start_logits + output_2.start_logits + output_3.start_logits + output_4.start_logits + output_5.start_logits + output_6.start_logits)/5
    end_logits = (output_1.end_logits + output_2.end_logits + output_3.end_logits + output_4.end_logits + output_5.end_logits + output_6.end_logits)/5
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position

        mask = data[1][0][k].bool() &  data[2][0][k].bool() # token type & attention mask
        masked_output_start = torch.masked_select(start_logits[k], mask.to(device))[:-1] # -1 is [SEP]
        n_best_start_index = _get_best_indexes(masked_output_start, n_best)
        masked_output_end = torch.masked_select(end_logits[k], mask.to(device))[:-1] # -1 is [SEP]
        n_best_end_index = _get_best_indexes(masked_output_end, n_best)


        masked_data = torch.masked_select(data[0][0][k], mask)[:-1]
        for i in n_best_start_index:
          for j in n_best_end_index:
            if i > j :
              continue
            elif ((masked_output_start[i] + masked_output_end[j]) > best_of_n_best_prob) & ((j - i)<=40):
              best_of_n_best_prob = masked_output_start[i] + masked_output_end[j]
              start_index_in_paragraph = i + doc_stride*k
              end_index_in_paragraph = j + doc_stride*k
              answer = tokenizer.decode(masked_data[i : j + 1])

    if ('[UNK]' in answer) & (is_test == False):
      start = dev_paragraphs_tokenized['offset_mapping'][paragraph_index][start_index_in_paragraph][0]
      end = dev_paragraphs_tokenized['offset_mapping'][paragraph_index][end_index_in_paragraph][1]
      answer = dev_paragraphs[paragraph_index][start:end].replace('✔', ' ').replace('✦','\u200b').replace('☺','\u200e').replace('☆','\u3000').replace('●','#')
    elif ('[UNK]' in answer) & (is_test == True):
      start = test_paragraphs_tokenized['offset_mapping'][paragraph_index][start_index_in_paragraph][0]
      end = test_paragraphs_tokenized['offset_mapping'][paragraph_index][end_index_in_paragraph][1]
      answer = test_paragraphs[paragraph_index][start:end].replace('✔', ' ').replace('✦','\u200b').replace('☺','\u200e').replace('☆','\u3000').replace('●','#')


    # print(f'final answer: {answer}')
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    if ('「' == answer[0]) & ('」' != answer[-1]):
      answer = answer + '」'
    if ('」' == answer[-1]) & ('「' != answer[0]):
      answer = '「' + answer
    if ('『' == answer[0]) & ('』' != answer[-1]):
      answer = answer + '』'
    if ('』' == answer[-1]) & ('『' != answer[0]):
      answer = '『' + answer
    
    return answer.replace(' ','')

    



In [9]:
def _get_best_indexes(logits, n_best_size):
    """Get the n-best logits from a list."""
    index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

    best_indexes = []
    for i in range(len(index_and_score)):
        if i >= n_best_size:
            break
        best_indexes.append(index_and_score[i][0])
    return best_indexes


In [11]:
print("Evaluating Test Set ...")
# tokenizer = BertTokenizerFast.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large")
result = []
n_best = 20

model_1.eval()
model_2.eval()
model_3.eval()
model_4.eval()
model_5.eval()
model_6.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output_1 = model_1(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output_2 = model_2(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output_3 = model_3(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output_4 = model_4(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output_5 = model_5(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output_6 = model_5(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output_1, output_2, output_3, output_4, output_5, output_6, test_questions[i]["paragraph_id"], 64, is_test = True, n_best = n_best))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

Completed! Result is in result.csv


In [12]:
from google.colab import files
files.download('./result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>